In [93]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

import os

root = None
repository = None
label = None
run_button = None

def update_label():
    label.config(text=f"Running energibridge...", foreground="#4CAF50")  # Green text
    
def find_gradle_build(folder):
    for root, _, files in os.walk(folder):
        if "build.gradle" in files or "build.gradle.kts" in files:
            return os.path.join(root, "build.gradle") if "build.gradle" in files else os.path.join(root, "build.gradle.kts")
    return None
    
def browse_folder():
    global repository
    folder_selected = filedialog.askdirectory()
    if folder_selected:
        gradle_file = find_gradle_build(folder_selected)
        if gradle_file:
            label.config(text=f"Found: {gradle_file}")
            repository = folder_selected
            run_button.config(state='normal')  
        else:
            label.config(text="gradle.build file not found")
            repository = None
            run_button.config(state='disabled')
            
            
def runBridge():
    
    update_label()
    
    

def main():
    global root, label, repository, run_button
    # Main window
    root = tk.Tk()
    root.title("GradleBridge")
    #root.geometry("800x500")
    root.configure(bg="#f8f9fa")

    #Style
    style = ttk.Style()
    style.theme_use("clam")  # Alternative themes: "alt", "default", "classic"
    style.configure("TFrame", background="#f8f9fa")
    style.configure("TLabel", font=("Arial", 14, "bold"), background="#f8f9fa", foreground="#333")
    style.configure("TEntry", font=("Arial", 12), padding=5)
    style.configure("run.TButton", font=("Arial", 12, "bold"), background="#4CAF50", foreground="white", padding=10)
    #style.map("run.TButton", background=[("active", "#45a049")]) 
    style.map("run.TButton", background=[("disabled", "#e0e0e0"), ("active", "#45a049")], foreground=[("disabled", "gray")])

    style.configure("browse.TButton", font=("Arial", 12, "bold"), background="#2196F3", foreground="white", padding=10)
    style.map("browse.TButton", background=[("active", "#1976D2")])  # Darker blue when hovered
    

    main_frame = ttk.Frame(root, style="TFrame", padding=20)
    main_frame.pack(expand=True, fill="both", anchor="center")

    #Entries (fields)    
    ttk.Label(main_frame, text="Iterations:", style="TLabel").pack(pady=5)
    iterations_entry = ttk.Entry(main_frame, style="TEntry")
    iterations_entry.pack(pady=5)

    ttk.Label(main_frame, text="Sleep (seconds):", style="TLabel").pack(pady=5)
    sleep_entry = ttk.Entry(main_frame, style="TEntry")
    sleep_entry.pack(pady=5)

    ttk.Label(main_frame, text="Interval (seconds):", style="TLabel").pack(pady=5)
    interval_entry = ttk.Entry(main_frame, style="TEntry")
    interval_entry.pack(pady=5)

    #Buttons
    browse_button = ttk.Button(main_frame, text="Select Folder", command=browse_folder, style="browse.TButton")
    browse_button.pack(side=tk.LEFT, expand=True)
    
    run_button = ttk.Button(main_frame, text="Run", command=runBridge, style="run.TButton", state='disabled')
    run_button.pack(side=tk.LEFT, expand=True)

    
    
    label = ttk.Label(main_frame, text="", style="TLabel")
    label.pack(pady=20)

    root.mainloop()

if __name__ == "__main__":
    main()
